In [ ]:
config_list = [
    {
        "model": "gpt-4-1106-preview",
        "api_key": "openai_api",
    }
]

llm_config_proxy = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 600
}

In [ ]:
llm_config_assistant = {
    "Seed" : 42,
    "temperature": 0,
        "functions": [
        {
            "name": "answer_PDF_question",
            "description": "Answer any PDF related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to PDF",
                    }
                },
                "required": ["question"],
            },
            
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}

In [ ]:
assistant = autogen.AssistantAgent(
            name="assistant",
            llm_config=llm_config_assistant,
            system_message="""You are a helpful assistant, Answer the question 
                              based on the context. Keep the answer accurate. 
                              Respond "Unsure about answer" if not sure about 
                              the answer."""
            
        )